# Capstone project - Applied Data Science

## Week 3 - Assignment Part 2

Welcome to Capstone Project for Applied Data Science course on coursera powered by IBM.

This notebook is for the Assignment in Week 3 Part 2

The goal of the assignment is to read the Totonto city postal codes from Wikipedia and create a dataframe ready for clustering assignment in Part 3 and add Geospatial data to the dataframe.

The Geospatial data is lattitude and longitude of the postal code in the data frame

I will try to usewe Geocoder Python package. 

The problem with this Package is you have to be persistent sometimes, in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates.

If I am unable to get teh coordinates, I will use the instructor given coordinates in a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data


In [40]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np

In [48]:
import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Prepare Wikipedia URL to read postal codes for Totonto city  
Use Beautiful Soup to read the webpage

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Read the contents of webpage using GET
r = requests.get(url)
# Create Beautiful Soup object 
soup = BeautifulSoup(r.content,'html5lib')

In [3]:
# Some global variables
postcode = ''
borough = ''
neighborhood = ''

Create Blank data frame with required columns

In [4]:
column_names = ['Postcode','Borough','Neighborhoods']
tor_post_codes = pd.DataFrame(columns=column_names)

Now, loop through the table of postal codes and append data frame

In [5]:
# Get the table object 
table = soup.find_all('table',{'class':'wikitable sortable'})

for tab in table:
# get the row of the table body
    for row in tab.tbody.find_all('tr'):
        try:
            # read cells 
            cells = row.find_all('td')

            postcode = cells[0].text
            # Some cells have hyperlink. The <a href> tag is present for them
            try:
                borough = cells[1].a.text
            except:
            # others do not have <a> tag    
                borough = cells[1].text
            
            neighborhood = cells[2].text.strip('\n')
            # if Neighbourhood is 'Not Assigned' neighbourhood = borough
            if neighborhood == 'Not assigned':
                neighborhood = borough 
                
            # keep only entries that are not 'Not assigned'
            if borough != 'Not assigned':
                tor_post_codes = tor_post_codes.append( {
                    'Postcode':postcode,
                    'Borough': borough,
                    'Neighborhoods': neighborhood
                }, ignore_index = True)
        except:
            print('Ignore Headers')

# inspect the newly created data frame
tor_post_codes.head()

Ignore Headers


,Postcode,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma

Use groupby function to group neighbourhoods

In [6]:
tor_post_codes = tor_post_codes.groupby(['Postcode','Borough'])['Neighborhoods'].apply(', '.join).reset_index()

tor_post_codes.head()

,Postcode,Borough,Neighborhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
tor_post_codes.shape

(103, 3)

An attempt to use the Geocoder Python package resulted in getting **None** for all the postal codes.  
So using the CSV fine provided by instructors

In [8]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
tot_post_coordinate = tor_post_codes.join(df_coordinates.set_index('Postal Code'), on='Postcode')
tot_post_coordinate.head()

,Postcode,Borough,Neighborhoods,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [11]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tot_post_coordinate['Latitude'], tot_post_coordinate['Longitude'], 
                                           tot_post_coordinate['Borough'], tot_post_coordinate['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Get the Borough in Toronto only. Search from the dataframe where borough has word Toronto in it

In [12]:
df_toronto = tot_post_coordinate[tot_post_coordinate['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,Postcode,Borough,Neighborhoods,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [13]:
# create map of Toronto City center using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhoods']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Now, lets take one neighbourhood and explore the venues there. We will use Foursquare data of nearby 100 interesting places around one venue

In [14]:
# Lets look around University of Toronto, Post code is M5S
df_uot = df_toronto.loc[df_toronto['Postcode']=='M5S'].reset_index(drop=True)
df_uot

,Postcode,Borough,Neighborhoods,Latitude,Longitude
0,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049


In [15]:
uot_lat = df_uot.loc[0,'Latitude']
uot_lon = df_uot.loc[0,'Longitude']
uot_name = df_uot.loc[0,'Neighborhoods']


print('Latitude and longitude values of {} are {}, {}.'.format(uot_name, 
                                                               uot_lat, 
                                                               uot_lon))

Latitude and longitude values of Harbord, University of Toronto are 43.6626956, -79.4000493.


In [16]:
# Get the Foursquare credentials
CLIENT_ID = 'JRHV2F5NFMSJ3VRLUQ50LOHIN30Q3S1QX40HZBVFYQ5WICUL' # Foursquare ID
CLIENT_SECRET = 'YQGRGPOYH2XFYUC0BIY3TF4FDHVUIDMPXBXO1FQ5FVE4VNNJ' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

# limit of number of venues returned by Foursquare API
LIMIT = 100
radius = 500 # define radius

In [17]:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    uot_lat, 
    uot_lon, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JRHV2F5NFMSJ3VRLUQ50LOHIN30Q3S1QX40HZBVFYQ5WICUL&client_secret=YQGRGPOYH2XFYUC0BIY3TF4FDHVUIDMPXBXO1FQ5FVE4VNNJ&v=20180605&ll=43.6626956,-79.4000493&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8b657cf594df7cd9de2861'},
 'response': {'headerLocation': 'University of Toronto',
  'headerFullLocation': 'University of Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 34,
  'suggestedBounds': {'ne': {'lat': 43.6671956045, 'lng': -79.39384042790832},
   'sw': {'lat': 43.6581955955, 'lng': -79.4062581720917}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5362c366498e602fbe1db395',
       'name': 'Yasu',
       'location': {'address': '81 Harbord St.',
        'lat': 43.66283719650635,
        'lng': -79.40321739973975,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.66283719650635,
          'lng': -79.40321739973975}],
        'distance': 255,
        'postalCode': 'M5S 1G

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,Almond Butterfly,Bakery,43.662836,-79.403365
3,Piano Piano,Italian Restaurant,43.662949,-79.402898
4,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
tor_downtown_venues = getNearbyVenues(names=df_toronto['Neighborhoods'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
#since Venue Category have name Neighborhood, we will have to rename column 'Neighborhood' to 'Neighborhoods'
tor_downtown_venues

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
5,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
9,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


In [24]:
print(tor_downtown_venues.shape)
tor_downtown_venues.head()

(1688, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [29]:
tor_downtown_venues.rename(columns={'Neighborhood':'Neighborhoods'}, inplace = True)

In [30]:
tor_downtown_venues.groupby('Neighborhoods').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhoods,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",98,98,98,98,98,98
Christie,16,16,16,16,16,16


In [31]:
print('There are {} uniques categories.'.format(len(tor_downtown_venues['Venue Category'].unique())))

There are 237 uniques categories.


Now, lets analyze neigbhorhoods using the Venue category.  
We will cluster teh neighborhoods using the venue category and plot the results

In [32]:
# One hot encode Venue category 
tor_onehot = pd.get_dummies(tor_downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# Add back the Neighborhood column
tor_onehot['Neighborhoods'] = tor_downtown_venues['Neighborhoods']

In [33]:
# Bring Neighborhood column to the front
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
fixed_columns

['Neighborhoods',
 'Afghan Restaurant',
 'Airport',
 'Airport Food Court',
 'Airport Gate',
 'Airport Lounge',
 'Airport Service',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Aquarium',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Workshop',
 'BBQ Joint',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Stadium',
 'Basketball Stadium',
 'Beach',
 'Beer Bar',
 'Beer Store',
 'Belgian Restaurant',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bubble Tea Shop',
 'Building',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Camera Store',
 'Caribbean Restaurant',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Church',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Arts Building',
 'College Gym',
 'College Rec Cente

In [34]:
tor_onehot = tor_onehot[fixed_columns]
tor_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
tor_onehot.shape

(1688, 238)

In [36]:
tor_groupped = tor_onehot.groupby('Neighborhoods').mean().reset_index()
tor_groupped

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,...,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.00,0.012195
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.051020,0.000000,0.051020,0.010204,0.000000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,...,0.000000,0.00,0.000000,0.012048,0.012048,0.000000,0.012048,0.012048,0.00,0.012048


In [37]:
tor_groupped.shape

(38, 238)

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
return_most_common_venues(tor_groupped.iloc[0, :], 10)

array(['Coffee Shop', 'Café', 'Thai Restaurant', 'Bar', 'Steakhouse',
       'Burger Joint', 'Hotel', 'Bakery', 'Sushi Restaurant',
       'American Restaurant'], dtype=object)

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = tor_groupped['Neighborhoods']

for ind in np.arange(tor_groupped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_groupped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Burger Joint,Hotel,Bakery,Sushi Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Bakery,Restaurant,Café,Farmers Market,Seafood Restaurant,Italian Restaurant,Cheese Shop
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Performing Arts Venue,Burrito Place,Stadium,Caribbean Restaurant,Bar,Furniture / Home Store,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Recording Studio,Farmers Market,Spa,Comic Shop,Skate Park,Burrito Place,Light Rail Station,Restaurant,Auto Workshop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Boat or Ferry,Sculpture Garden


In [63]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_groupped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [64]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       1, 0, 3, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df_toronto

tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')

tor_merged.head()

,Postcode,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Pub,Coffee Shop,Other Great Outdoors,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Gym,Italian Restaurant,Coffee Shop,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhoods'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now that we have clustered the venues, lets examine them 

### **Cluster 1**

In [54]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Neighborhood,Pub,Coffee Shop,Other Great Outdoors,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
2,East Toronto,0,Gym,Italian Restaurant,Coffee Shop,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery,Fast Food Restaurant
3,East Toronto,0,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Brewery,Seafood Restaurant
5,Central Toronto,0,Sandwich Place,Breakfast Spot,Gym,Park,Hotel,Food & Drink Shop,Burger Joint,Asian Restaurant,Eastern European Restaurant,Doner Restaurant
6,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Park,Sandwich Place,Salon / Barbershop,Rental Car Location,Chinese Restaurant,Gym / Fitness Center
7,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Thai Restaurant,American Restaurant,Pharmacy
9,Central Toronto,0,Pub,Convenience Store,Coffee Shop,American Restaurant,Bagel Shop,Sushi Restaurant,Supermarket,Sports Bar,Light Rail Station,Medical Center
11,Downtown Toronto,0,Coffee Shop,Park,Restaurant,Pizza Place,Bakery,Pub,Italian Restaurant,Café,Market,Beer Store
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Fast Food Restaurant,Pub,Bubble Tea Shop,Café


So, the first cluster, cluster 0 has more coffee shops as top venues. they offer more variety to office going people. The sandwitch shops, Pizza places and Lounges offer great variety to blue coller workers working in offices in downtown

### **Cluster 2**

In [55]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,1,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


There is only one neighborhood in this cluster offering parks and swimming pools. So, family oriented people can opt for this neighborhood.

### **Cluster 3**

In [56]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Home Service,Ice Cream Shop,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


This cluster is similar to the one above. Families can look at the options to stay there

### **Cluster 4**

In [57]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Restaurant,Playground,Tennis Court,Summer Camp,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store


This neighborhood offers more options to senior folks. 

### **Cluster 5**

In [58]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
23,Central Toronto,4,Jewelry Store,Trail,Park,Sushi Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio


This cluster is similar to the Cluster 2 and 3 above. Families can look at the options to stay there